In [71]:
import pytesseract
from pytesseract import Output
import csv
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
import cv2
import numpy as np
from matplotlib import pyplot as plt
img = cv2.imread('1.JPG')
height, width, channel = img.shape

def imshow(img, label='default'):
    cv2.imshow(label, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
img = cv2.resize(img, (width*2, height*2))

alpha = 0.9
beta = 0
adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
adjusted = cv2.resize(adjusted, (width*2, height * 2))

height, width, channel = img.shape


# Convert image to grey scale, erode horizontally and draw horizontal line
gray_image = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)
thresh_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]


kernel = np.ones((1,30), np.uint8)  #for horizontal erosion
e_im = cv2.erode(gray_image, kernel, iterations=2)  #performing erosion



edges = cv2.Canny(e_im,0,150,apertureSize = 5)     #edges

minLineLength=333
#houghlines for detecting those erosion lines
lines = cv2.HoughLinesP(image=edges,rho=0.5,theta=np.pi/180, threshold=222,lines=np.array([]), minLineLength=minLineLength,maxLineGap=7) 
a,b,c = lines.shape
# Draw the lines
if lines is not None:
    for i in range(0, len(lines)):
        l = lines[i][0]
        cv2.line(thresh_img, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)

imshow(thresh_img)

#New kernel for Vertical erode and line
kernel = np.ones((500,12), np.uint8)  #for horizontal erosion
e_im = cv2.erode(gray_image, kernel, iterations=4)  #performing erosion
#imshow(e_im)

edges = cv2.Canny(e_im,100,0,apertureSize = 3)
#imshow(edges)


lines = cv2.HoughLinesP(image=edges,rho=0.5,theta=np.pi/180, threshold=222,lines=np.array([]), minLineLength=minLineLength,maxLineGap=7) 
a,b,c = lines.shape
# Draw the lines
if lines is not None:
    for i in range(0, len(lines)):
        l = lines[i][0]
        cv2.line(thresh_img, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)

imshow(thresh_img)




In [72]:
# Create computer vision

custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'
d = pytesseract.image_to_data(thresh_img, config=custom_config, output_type=Output.DICT)
df = pd.DataFrame(d)

# clean up blanks
df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# sort blocks vertically
sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
for block in sorted_blocks:
    curr = df1[df1['block_num'] == block]
    sel = curr[curr.text.str.len() > 3]
    # sel = curr
    char_w = (sel.width / sel.text.str.len()).mean()
    prev_par, prev_line, prev_left = 0, 0, 0
    text = ''
    for ix, ln in curr.iterrows():
        
        # add new line when necessary
        if prev_par != ln['par_num']:
            text += '\n'
            prev_par = ln['par_num']
            prev_line = ln['line_num']
            prev_left = 0
        elif prev_line != ln['line_num']:
            text += '\n'
            prev_line = ln['line_num']
            prev_left = 0

        added = 0  # num of spaces that should be added
        if ln['left'] / char_w > prev_left + 1:
            added = int((ln['left']) / char_w) - prev_left
            text += ' ' * added
        text += ln['text'] + ' '
        prev_left += len(ln['text']) + added + 1
        
    text += '\n'
    print(text)
    
    check = list(text.split("\n"))
   


      Nikhil     Honda Cars      Manager Project    IT              nikhilarora@hondacarindia.com J 9811745280 
  Arora          India Ltd       Manager IT 
  &  Anil        New  Holland    Manager Deputy     IT              anil.patnaik@cnh.com            9999898176 
  Patnaik        Fiat            Manager                                                            / 9109134400 
  @&  Mr.        Paytm           vP                 Information     priyank.nag@paytmbank.com       9810566462 
  Priyank Nag    Payment Bank                       Technology 
  @   Pankaj     GFL Ltd.        GM  GM             IT              pshrivastava@gfl.co.in          012 
     .                           Information                                                        06149600 
  Shrivastava 
                                 Technology                                                         Ext:966 
  &  Aftab       Mahagun         Manager            IT              aahmad@mahagunindia.com         9

In [73]:
#Now read junk data


output_text = []
test = []
custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'


# Get all OCR output information from pytesseract
ocr_output_details = pytesseract.image_to_data(thresh_img, output_type = Output.DICT, config=custom_config, lang='eng')
# Total bounding boxes
n_boxes = len(ocr_output_details['text'])

# Extract and draw rectangles for all bounding boxes
for i in range(n_boxes):
    (x, y, w, h) = (ocr_output_details['left'][i], ocr_output_details['top'][i], ocr_output_details['width'][i], ocr_output_details['height'][i])
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Print OCR Output
print(ocr_output_details.keys())

# Arrange output text from OCR into the format as per image

word_list = []
last_word = []

for word in ocr_output_details['text']:
    # If there is no text in an element
    if word != '':
        word_list.append(word)
        last_word = word
    # Append final list of words with valid words
    if (last_word != '' and word == '') or (word == ocr_output_details['text'][-1]):
        output_text.append(word_list)
        test.append(word_list)
        word_list = []



dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [67]:
output_text

[[],
 [],
 [],
 [],
 [' Nischal',
  'Tv',
  'Today',
  'Manager',
  'Information',
  'nischal.maheshwari@intoday.com',
  '9818444340'],
 ['Maheshwari', 'Network', 'Technology'],
 [],
 ['@',
  'Sandeep',
  'Zee',
  'Media',
  'Manager',
  'Information',
  'sandeep.mishra@zeemedia.esselgroup.com',
  '9811417694'],
 ['Mishra', 'Limited.', 'Technology'],
 [],
 ['@', 'Prashant', 'Manager', 'IT', 'prashant.bhardwaj@flexfilm.com', '0'],
 ['Bhardwaj', 'Manager', '-', 'IT', '9873912930'],
 ['Ravi',
  'Uflex',
  'GM',
  'Sr',
  'GM',
  'IT',
  'ravi.ramakrishnan@flexfilm.com',
  '9891001443'],
 ['Ramakrishnan', 'Global-IT']]

In [46]:
check

['',
 '   Nischal       Tv Today        Manager            Information    nischal.maheshwari@intoday.com           9818444340 ',
 '  Maheshwari     Network                            Technology ',
 '  @  Sandeep     Zee Media       Manager            Information    sandeep.mishra@zeemedia.esselgroup.com   9811417694 ',
 '  Mishra         Limited.                           Technology ',
 '  @  Prashant                    Manager            IT             prashant.bhardwaj@flexfilm.com           0 ',
 '  Bhardwaj                       Manager - IT                                                               9873912930 ',
 '  Ravi           Uflex           GM  Sr GM          IT             ravi.ramakrishnan@flexfilm.com           9891001443 ',
 '  Ramakrishnan                   Global-IT ',
 '']

In [74]:



def get_column(check, word, start):
    global low_row
    word = word.strip()
    
    flag = False
    for i, check_line in enumerate(check[start:]):
    
        if check_line:
            try:
                if not word in check_line:
                    print(".")
                else:
                    fetched_pos = check_line.lower().find(word.lower())
                    print("fetched pos ", fetched_pos)
                    flag = True
                    print(i)
                    if fetched_pos < 16:
                        column = 0
                    elif fetched_pos < 33:
                        column = 1
                    elif fetched_pos < 52:
                        column = 2
                    elif fetched_pos < 67:
                        column = 3
                    elif fetched_pos < 108 and "@" in word:
                        column = 4
                    else:
                        column = 5 
                    
                    print(f"{word} is in row: {low_row}, column: {column}")
                    return [i, column]
            except UnboundLocalError:
                print("THIS Could not find word.")
                return -1
    if not flag:
        print(f"Did not find \n {word}")
        return [flag,flag]
    else:
        low_row+= 1
   
                    

In [70]:
row = [0 for x in range(7)]
row

[0, 0, 0, 0, 0, 0, 0]

In [75]:
#Look through lens

all_row = []
low_row = 0
for line in output_text:
    print(f"Line has {len(line)} words.")

    if line:
        print(line)
        row = [0 for x in range(7)]
        for idx, word in enumerate(line):
            print(idx,word)
            if word == '@' or word == '&' or word == "." or word == "@&":
                continue
            pos = get_column(check, word, low_row)
            row_num = pos[0]
            column = pos[1]
            print(column)
            
            if not column and not column == 0 :
                #low_row = low_row.append(line)
                break
                
            elif type(row[column]) == int:
                row[column] = word
            elif type(row[column]) == str:
                row[column]+= ' '+word
      
    
        print(row)
        row.append(row_num)
        all_row.append(row)
            
    else:
        print(line)
            
        

Line has 0 words.
[]
Line has 0 words.
[]
Line has 0 words.
[]
Line has 0 words.
[]
Line has 9 words.
['Nikhil', 'Honda', 'Cars', 'Manager', 'Project', 'IT', 'nikhilarora@hondacarindia.com', 'J', '9811745280']
0 Nikhil
fetched pos  6
1
Nikhil is in row: 0, column: 0
0
1 Honda
fetched pos  17
1
Honda is in row: 0, column: 1
1
2 Cars
fetched pos  23
1
Cars is in row: 0, column: 1
1
3 Manager
fetched pos  33
1
Manager is in row: 0, column: 2
2
4 Project
fetched pos  41
1
Project is in row: 0, column: 2
2
5 IT
fetched pos  52
1
IT is in row: 0, column: 3
3
6 nikhilarora@hondacarindia.com
fetched pos  68
1
nikhilarora@hondacarindia.com is in row: 0, column: 4
4
7 J
fetched pos  44
1
J is in row: 0, column: 2
2
8 9811745280
fetched pos  100
1
9811745280 is in row: 0, column: 5
5
['Nikhil', 'Honda Cars', 'Manager Project J', 'IT', 'nikhilarora@hondacarindia.com', '9811745280', 0]
Line has 5 words.
['Arora', 'India', 'Ltd', 'Manager', 'IT']
0 Arora
.
fetched pos  2
2
Arora is in row: 0, column

In [49]:
for r in all_row:
    print(*r, sep="\t")

 Nischal	Tv Today	Manager	Information	nischal.maheshwari@intoday.com	9818444340	0	1
Maheshwari	Network	0	Technology	0	0	0	2
Sandeep	Zee Media	Manager	Information	sandeep.mishra@zeemedia.esselgroup.com	9811417694	0	3
Mishra	Limited.	0	Technology	0	0	0	2
Prashant	0	Manager	IT	prashant.bhardwaj@flexfilm.com	0	0	1
Bhardwaj	0	Manager -	IT	0	9873912930	0	6
Ravi	Uflex	GM Sr GM	IT	ravi.ramakrishnan@flexfilm.com	9891001443	0	7
Ramakrishnan	0	Global-IT	0	0	0	0	8


In [77]:
test = pd.DataFrame(all_row)
test

,0,1,2,3,4,5,6,7
0,Nikhil,Honda Cars,Manager Project J,IT,nikhilarora@hondacarindia.com,9811745280,0,1
1,Arora,India Ltd,Manager,IT,0,0,0,1
2,Anil,New Holland,Manager Deputy,IT,anil.patnaik@cnh.com,9999898176,0,3
3,Patnaik,Fiat,Manager,0,0,/ 9109134400,0,4
4,Mr.,Paytm,vP,Information,priyank.nag@paytmbank.com,9810566462,0,5
5,Priyank Nag,Payment Bank,0,Technology,0,0,0,6
6,Pankaj,GFL Ltd.,GM GM,IT,pshrivastava@gfl.co.in,012,0,7
7,0,0,0,Information,0,06149600,0,8
8,Shrivastava,0,0,0,0,0,0,9
9,0,0,0,Technology,0,Ext:966,0,10


In [78]:
test = pd.DataFrame(all_row)
test.columns = ['Name','Company','Designation','Department','Email','Phone','Score','Row']
test['Name'] = test['Name'].replace(0, np.nan)
test['Name'].ffill(inplace =True)

test

,Name,Company,Designation,Department,Email,Phone,Score,Row
0,Nikhil,Honda Cars,Manager Project J,IT,nikhilarora@hondacarindia.com,9811745280,0,1
1,Arora,India Ltd,Manager,IT,0,0,0,1
2,Anil,New Holland,Manager Deputy,IT,anil.patnaik@cnh.com,9999898176,0,3
3,Patnaik,Fiat,Manager,0,0,/ 9109134400,0,4
4,Mr.,Paytm,vP,Information,priyank.nag@paytmbank.com,9810566462,0,5
5,Priyank Nag,Payment Bank,0,Technology,0,0,0,6
6,Pankaj,GFL Ltd.,GM GM,IT,pshrivastava@gfl.co.in,012,0,7
7,Pankaj,0,0,Information,0,06149600,0,8
8,Shrivastava,0,0,0,0,0,0,9
9,Shrivastava,0,0,Technology,0,Ext:966,0,10


In [52]:
test['Name'] = test['Name'].replace(0, np.nan)
test['Name'].ffill(inplace =True)
test


,Name,Company,Designation,Department,Email,Phone,Score,Row
0,Nischal,Tv Today,Manager,Information,nischal.maheshwari@intoday.com,9818444340,0,1
1,Maheshwari,Network,0,Technology,0,0,0,2
2,Sandeep,Zee Media,Manager,Information,sandeep.mishra@zeemedia.esselgroup.com,9811417694,0,3
3,Mishra,Limited.,0,Technology,0,0,0,2
4,Prashant,0,Manager,IT,prashant.bhardwaj@flexfilm.com,0,0,1
5,Bhardwaj,0,Manager -,IT,0,9873912930,0,6
6,Ravi,Uflex,GM Sr GM,IT,ravi.ramakrishnan@flexfilm.com,9891001443,0,7
7,Ramakrishnan,0,Global-IT,0,0,0,0,8


In [ ]:
for _ in names:
    row_df = 